<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso2/ciclo4/M5U4_Redes_Neuronales_Recurrentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://drive.google.com/uc?export=view&id=1VV2e_u46fNm_ewns8QW2HGRZAPHh-e2t" alt = "Encabezado MLDS" width = "100%">  </img>

# **Modelos de Lenguaje**
----
Los modelos de lenguaje son herramientas fundamentales en el campo del procesamiento del lenguaje natural (NLP, por sus siglas en inglés). Estos modelos permiten a las máquinas comprender y generar texto de manera más efectiva, lo que ha llevado a importantes avances en aplicaciones como traducción automática, resumen de texto, análisis de sentimientos y chatbots, entre otros. En este notebook, exploraremos los conceptos básicos de los modelos de lenguaje, los diferentes niveles en los que pueden operar y cómo se pueden implementar utilizando redes neuronales.

## Definición de modelo de lenguaje

Un modelo de lenguaje es un sistema matemático que permite estimar la probabilidad de una secuencia de palabras o caracteres en un texto. El objetivo principal es capturar las estructuras y patrones presentes en el lenguaje natural para generar texto coherente y gramaticalmente correcto.

## Niveles de modelos de lenguaje

* Modelos a nivel de carácter: Estos modelos tratan el texto como una secuencia de caracteres individuales y estiman la probabilidad de cada carácter dado el contexto de los caracteres anteriores. Aunque son más simples, tienden a ser menos eficientes en términos de rendimiento y tiempo de entrenamiento.

* Modelos a nivel de palabra: A diferencia de los modelos a nivel de carácter, estos modelos consideran el texto como una secuencia de palabras y estiman la probabilidad de cada palabra dado el contexto de las palabras anteriores. Estos modelos son más eficientes y producen resultados de mayor calidad.

## N-gramas

Los n-gramas son una técnica común en el modelado de lenguaje que consiste en dividir el texto en fragmentos de n palabras o caracteres consecutivos. Por ejemplo, un bigrama (n=2) es una secuencia de dos palabras o caracteres, mientras que un trigrama (n=3) es una secuencia de tres palabras o caracteres. Los modelos de n-gramas estiman la probabilidad de una palabra o carácter en función de las n-1 palabras o caracteres anteriores. Aunque los n-gramas pueden capturar cierta información contextual, su capacidad para modelar dependencias a largo plazo es limitada.

##  Redes neuronales en el modelado de lenguaje

Las redes neuronales han demostrado ser una herramienta poderosa para el modelado de lenguaje, ya que pueden capturar relaciones más complejas y dependencias a largo plazo en el texto. Algunos de los enfoques más populares incluyen:

*    Redes neuronales recurrentes (RNN): Las RNN son una clase de redes neuronales que pueden procesar secuencias de datos de longitud variable. Son especialmente adecuadas para el modelado de lenguaje, ya que pueden mantener información sobre el contexto previo en su estado interno. Las variantes más avanzadas, como las LSTM (Long Short-Term Memory) y las GRU (Gated Recurrent Unit), han demostrado un rendimiento aún mejor en tareas de NLP.

*    Modelos de atención y _Transformers_: Los modelos basados en mecanismos de atención, como el modelo _Transformer_, han revolucionado el campo del NLP. Estos modelos son capaces de capturar dependencias a largo plazo y contextualizar cada palabra o carácter en función de su posición y contexto en la secuencia en la secuencia de entrada. Los _Transformers_ han demostrado un rendimiento superior en una amplia gama de tareas de NLP y han sido la base para modelos de lenguaje de última generación, como BERT, GPT y T5.

*    Redes neuronales convolucionales (CNN): Aunque las CNN son más conocidas por su uso en la clasificación y detección de imágenes, también se han aplicado con éxito al modelado de lenguaje. En este contexto, las CNN pueden capturar patrones locales y características relevantes en secuencias de texto. Sin embargo, a menudo se combinan con otros enfoques, como las RNN o los _Transformers_, para lograr un mejor rendimiento.


En este notebook introduciremos las **Redes Neuronales Recurrentes**. Veremos:

- Modelos probabilísticos a nivel de carácter.
- Redes Neuronales Recurrentes Simples
- LSTM (_Long Short Term Memory_)
- GRU (_Gated Recurrent Unit_)
- Generación automática de texto


Comencemos entonces con un ejemplo de generación automática de texto. Para ello comenzamos importando las librerías necesarias:


In [ ]:
# Seleccionamos la versión más reciente de Tensorflow 2.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os, random
from pprint import pprint
%matplotlib inline
plt.style.use("ggplot")
# Seleccionamos una semilla para los RNG (Random Number Generator)
tf.random.set_seed(123)
np.random.seed(123)

# **1. Modelos probabilísticos del lenguaje a nivel de carácter**
----
Una de las principales ventajas de las redes neuronales es que **permiten aproximar modelos o distribuciones comúnmente usados como modelos probabilísticos del lenguaje**. Esto se consigue fundamentalmente al explotar las propiedades de algunos tipos de activaciones como _Sigmoide_ y _Softmax_.
- Por ejemplo, una familia de modelos probabilísticos típicos son **los modelos de n-gramas**, donde se modela una distribución $P(c_{k+1}|c_1,\dots,c_{k})$ que representa la probabilidad de que un carácter $c_k$ tenga lugar luego de un k-gram o secuencia de caracteres $c_1,\dots,c_{k}$.

El objetivo de una red neuronal en esta tarea es **aproximar esta distribución**, esto se hace típicamente de forma paramétrica (pesos de la red) y la salida está asociada comúnmente con el valor de la probabilidad. Veamos un ejemplo simple de esto con un modelo 3-gramas en un corpus muy pequeño.

- Comenzamos definiendo el corpus a partir de un texto de ejemplo tomado de la biografía de Alan Turing de Wikipedia :


In [ ]:
corpus = """
alan mathison turing es considerado uno de los padres de la ciencia de la
computacion y precursor de la informatica moderna. proporciono una influyente
formalizacion de los conceptos de algoritmo y computacion: la maquina de turing.
en el campo de la inteligencia artificial, es conocido sobre todo por la
concepcion del test de turing, un criterio segun el cual puede juzgarse
la inteligencia de una maquina si sus respuestas en la prueba son indistinguibles
de las de un ser humano."""

Ahora vamos a codificar cada uno de los caracteres como enteros. Más adelante veremos en detalle este proceso (llamado **vectorización**).

- Primero extraemos el vocabulario del ejemplo (a nivel de carácter) y calculamos su tamaño :


In [ ]:
example_vocab = sorted(set(corpus))
example_vocab_size = len(example_vocab)

Luego asignamos un número entero a cada carácter del vocabulario :

In [ ]:
char2int = {u:i for i, u in enumerate(example_vocab)}
int2char = np.array(example_vocab)

Veamos:

In [ ]:
# Mostramos la codificación de los primeros 20 caracteres
print('{')
for char,_ in zip(char2int, range(20)):
    print(f'  {repr(char):4s}: {char2int[char]:3d}')
print('  ...\n}')
print("Tamaño vocabulario:", len(example_vocab))

Convertimos el dataset a 3-gramas (secuencias de tres caracteres y el carácter sucesor) :

In [ ]:
# Definimos las variables donde se guardará el dataset
X = []; y = []
# Iteramos para todos los posibles 3-gramas en el corpus
for i in range(len(corpus)-4):
    X.append(corpus[i:i+3])
    y.append(corpus[i+3])
print(X[:5])
print(y[:5])

Veamos un ejemplo de secuencia de caracteres y el sucesor:

In [ ]:
print(f"3-grama ---> sucesor")
print(f"{X[1]}     ---> {y[1]}")
print(f"{X[15]}     ---> {y[15]}")
print(f"{X[17]}     ---> {y[17]}")

Ahora, convertimos los datos a valores enteros:

In [ ]:
X_int = np.array(list(map(lambda i: [char2int[i[0]], char2int[i[1]], char2int[i[2]]], X)))
y_int = np.array(list(map(lambda i: char2int[i], y)))
print(X_int.shape)
print(y_int.shape)

Veamos:

In [ ]:
print(f"3-grama ---> codificación")
print(f"{X[1]}     ---> {X_int[1]}")
print(f"{X[15]}     ---> {X_int[15]}")
print(f"{X[17]}     ---> {X_int[17]}")

Nuestra red neuronal tomará como entrada **la codificación en enteros** de los caracteres y la salida será la **probabilidad de cada uno de los posibles caracteres** de ser el sucesor.

- Definamos este modelo en _Tensorflow_: Será un modelo con una capa de entrada de 3 neuronas, seguido por una capa densa con activación _Sigmoide_, y una capa de salida de tamaño igual al tamaño del vocabulario

In [ ]:
input = tf.keras.layers.Input(shape=(3,))
densa = tf.keras.layers.Dense(64, activation='sigmoid')(input)
logits = tf.keras.layers.Dense(example_vocab_size, activation='linear')(densa)
probits = tf.keras.layers.Activation('softmax')(logits)

model = tf.keras.models.Model(inputs=input, outputs=probits)

In [ ]:
model.summary()

En este caso definimos dos capas llamadas **logits** y **probits**, los cuales son conceptos comunes en modelos probabilísticos.

- Por un lado, los **logits** se encuentran en un dominio real y representan una especie de log-verosimilitud (logaritmo de una probabilidad) que a su vez está relacionada con una distribución de probabilidad sobre caracteres.
- Por otro lado, los **probits** se encuentran en un rango de 0 a 1 y representan la probabilidad de cada carácter. **probits** se calcula a partir de **logits** usando la función de activación _softmax_.

Compilamos usando `categorical_crossentropy` como función de pérdida y `Adam` como optimizador con una tasa de aprendizaje de $0.01$:

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(learning_rate=1e-2))


Para entrenar el modelo necesitamos convertir las etiquetas o caracteres sucesores en *one-hot* :

In [ ]:
Y = tf.keras.utils.to_categorical(y_int, num_classes=example_vocab_size)
print(Y.shape)

Finalmente, entrenamos el modelo durante 1000 epochs:

In [ ]:
hist = model.fit(X_int, Y, epochs=1000)

Veamos un ejemplo de las probabilidades predichas para algunos 3-gramas. Revise cuál carácter es el más probable de ser el siguiente, según el modelo:

* **`ala`**

In [ ]:
# 3-gram ejemplo
seq = 'ala'
# Codificación en enteros
seq = np.array([char2int[i] for i in seq]).reshape(1, -1)
print(seq)
# Predicciones
probs = model.predict(seq)
# Veamos un diagrama de barras de las probabilidades
plt.figure(figsize=(8,8))
plt.bar(int2char, probs[0]);

* **`tur`**

In [ ]:
# 3-gram ejemplo
seq = 'tur'
# Codificación en enteros
seq = np.array([char2int[i] for i in seq]).reshape(1, -1)
# Predicciones
probs = model.predict(seq)
# Veamos un diagrama de barras de las probabilidades
plt.figure(figsize=(8,8))
plt.bar(int2char, probs[0]);

* **`aaa`**

In [ ]:
# 3-gram ejemplo
seq = 'aaa'
# Codificación en enteros
seq = np.array([char2int[i] for i in seq]).reshape(1, -1)
# Predicciones
probs = model.predict(seq)
# Veamos un diagrama de barras de las probabilidades
plt.figure(figsize=(8,8))
plt.bar(int2char, probs[0])

 # **2. Redes recurrentes**
----
Las redes neuronales recurrentes (RNN) son un tipo de arquitectura de redes neuronales especialmente diseñada para procesar secuencias de datos. Más específicamente, pueden recordar, y sus decisiones están influenciadas por el pasado. Esto las hace ideales para tareas que involucran datos secuenciales o temporales. Se utilizan en una amplia gama de aplicaciones y dominios, algunos de los cuales incluyen:

*    Predicción de series temporales: Las RNN pueden utilizarse para predecir valores futuros en series temporales, como el clima o las ventas de productos. Las RNN pueden modelar relaciones temporales y capturar patrones a lo largo del tiempo.

*    Reconocimiento de voz: Las RNN son empleadas en sistemas de reconocimiento automático de voz para convertir señales de audio en texto. Estos sistemas deben tener en cuenta la secuencia temporal de las señales de audio para generar transcripciones precisas.

*    Generación de música: Las RNN también pueden ser utilizadas para generar composiciones musicales, ya que la música es inherentemente secuencial y temporal. Las RNN pueden aprender patrones y estructuras en la música para crear nuevas piezas.

*    Control de robots y sistemas autónomos: En el control de robots y vehículos autónomos, las RNN pueden utilizarse para procesar secuencias de datos de sensores y tomar decisiones en función de la información contextual.

*    Análisis de vídeo: Las RNN pueden utilizarse en el análisis de vídeos para detectar y reconocer objetos, acciones y eventos a lo largo del tiempo, ya que los vídeos son secuencias de imágenes.

*    Y por supuesto, procesamiento del lenguaje natural: las RNN son ampliamente utilizadas en tareas de NLP, como traducción automática, análisis de sentimientos, generación de texto, resumen automático, reconocimiento de entidades nombradas y más. Estas tareas a menudo requieren modelar secuencias de palabras o caracteres y capturar dependencias de contexto.



A pesar de que las redes neuronales multicapa también recuerdan cosas, **las redes recurrentes utilizan información de lo que recuerdan de entradas anteriores para generar una salida**. Este comportamiento se consigue al añadir un lazo de realimentación sobre una determinada red neuronal $A$ como se muestra en la siguiente figura :

<center><img src='https://drive.google.com/uc?export=view&id=1MX2WYF6JGBZ5OsphTU-C2iAjJK7sQili' alt = "Gráfico ilustrativo del lazo de realimentación sobre una red neuronal " width ="25%"></img></center>

Las redes neuronales aprenden por medio de la propagación de un error, no obstante, debido a la realimentación no es posible entrenar las redes recurrentes directamente, por ello, se utiliza una **aproximación** al considerar solamente un número finito de estados previos, como se muestra a continuación :

<center><img src='https://drive.google.com/uc?export=view&id=10SLpNXUgeCx10RX9EQu7IGFlUeKhYiVs' alt ="Gráfico ilustrativo del número finito de estados previos en una red" width ="100%"></img></center>

Este tipo de arquitectura en forma de cadena es precisamente lo que hace que este tipo de redes sean muy útiles para datos con estructura secuencial.

En este caso, veremos la implementación de tres tipos de redes recurrentes comúnmente usadas :  
- ```tf.keras.layers.SimpleRNN```
-  ```tf.keras.layers.GRU ```
-  ```tf.keras.layers.LSTM ```.


## **Modelos secuenciales**
----

Cuando trabajamos con secuencias hay varias formas de representar los datos, no obstante, lo más típico es utilizar arquitecturas de tipo **_Many to One_** _(Muchos a uno)_ o **_Many to Many_** _(Muchos a Muchos)_ :

<center><img src='https://drive.google.com/uc?export=view&id=1ABK3maujqA-63sGn9F5-IiNpE8jTAjxh' alt ="Gráfico ilustrativo de las distintas arquitecturas de tipo Many to One y Many to Many"width ="100%"></img></center>

En nuestra aplicación de modelos probabilísticos, una arquitectura *Many-to-One* puede ser usada para predecir el siguiente carácter dada determinada oración o secuencia de caracteres : <center>$P(c_{k+1}|c_1,\dots,c_{k})$</center>

Una arquitectura *Many-to-Many* podría ser usada para predecir la siguiente oración o secuencia de caracteres:
<center>$P(c_{k+1}, \dots, c_{k+m}|c_1,\dots,c_{k})$. </center>

En los siguientes ejemplos trabajaremos a **nivel de carácter**. Utilizaremos una arquitectura de tipo *Many-to-Many*, es decir nuestro modelo recibirá una secuencia y predecirá una secuencia.


## **2.1. Procesamiento de los datos**
----

### **`Dataset` de _Tensorflow_**

En _TensorFlow_ existe un objeto de tipo `Dataset`. Un `Dataset` de TensorFlow es una abstracción de alto nivel que representa una secuencia de elementos, donde cada elemento consta de uno o más tensores. Los objetos `Dataset` facilitan la carga, el preprocesamiento y la iteración de datos en el proceso de entrenamiento y evaluación de modelos de aprendizaje automático.

Además, los `Dataset` están diseñados para ser eficientes y escalables, lo que permite cargar y procesar grandes cantidades de datos de manera eficiente, incluso en hardware limitado. Algunas de las características clave de los `Dataset` de _TensorFlow_ incluyen la capacidad de:

*    Realizar transformaciones de datos, como mapeo, filtrado, reducción y mezcla.
*    Cargar datos de diversas fuentes, como archivos locales, sistemas de archivos distribuidos y servicios en la nube.
*    Realizar la carga y el preprocesamiento de datos de manera paralela y asíncrona para aprovechar al máximo el hardware disponible.

### **Método `from_tensor_slices`**

El método `from_tensor_slices` es una función de la clase `Dataset` que permite crear un nuevo dataset a partir de tensores existentes. Esta función toma como entrada uno o más tensores y devuelve un dataset donde cada elemento corresponde a una "rebanada" de los tensores de entrada a lo largo de su primera dimensión.

La función `from_tensor_slices` es especialmente útil cuando se trabaja con datos estructurados, como imágenes y etiquetas, que se almacenan como tensores. Al utilizar este método, se pueden crear fácilmente datasets de entrenamiento y validación que contengan pares de datos y etiquetas correspondientes.

### **Ejemplo de uso del método `from_tensor_slices`**

Supongamos que tenemos dos tensores, `data` y `labels`, que representan un conjunto de imágenes y sus etiquetas correspondientes:



In [ ]:
data = tf.random.normal([100, 28, 28])  # 100 imágenes de 28x28
labels = tf.random.uniform([100], minval=0, maxval=10, dtype=tf.int32)  # 100 etiquetas

Podemos utilizar el método `from_tensor_slices` para crear un dataset a partir de estos tensores:

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))

Ahora, dataset es un objeto Dataset que contiene 100 elementos, donde cada elemento es un par de tensores (imagen, etiqueta).

### **Método `batch`**

El método `batch` es otra función importante de la clase `Dataset` en _TensorFlow_, que permite agrupar elementos consecutivos de un dataset en lotes (batches) de un tamaño específico.

- Ahora podemos aplicar el método batch para agrupar los elementos del dataset en lotes de un tamaño específico, por ejemplo, 32:

In [ ]:
batch_size = 32
dataset = dataset.batch(batch_size)

Después de aplicar el método `batch`, el `dataset` contendrá lotes de 32 elementos cada uno (excepto posiblemente el último _batch_, que podría ser más pequeño si el número total de elementos no es divisible por el tamaño del _batch_). Al iterar sobre este `dataset`, obtendremos tensores de forma `(batch_size, data_shape)` para los datos y las etiquetas.



**La Biblia**

Como ejemplo en este notebook utilizaremos un libro de acceso libre, cuyo texto tiene una estructura particular: la **Biblia**. Veamos cómo cargarlo y preprocesarlo.

Primero comenzamos cargando el texto y codificándolo en un formato típico de _Python_ desde un archivo de texto plano:

In [ ]:
!wget -O biblia.txt https://raw.githubusercontent.com/medelr/MDDS5/gh-pages/biblia.txt?raw=true
with open("biblia.txt", "rb") as f:
    text = f.read().decode(encoding="latin-1")
print(f"Longitud del texto en caracteres: {len(text)}")

Ahora veamos los primeros 1000 caracteres del texto :

In [ ]:
print(text[:1000])

Podemos hacer una **exploración descriptiva del texto**. Por ejemplo, empecemos contando el número total de caracteres diferentes. Al convertir el texto en un objeto tipo `set` se eliminan los duplicados y podemos contar los caracteres :

In [ ]:
# Obtenemos los caracteres únicos en el texto
vocab = sorted(set(text))
vocab_size = len(vocab)
print(f'Se encontraron {vocab_size} caracteres únicos')

Veamos:

In [ ]:
vocab

Ahora veamos cómo preparar el dataset utilizando ```tf.data```:

### **2.1.1. Vectorización**
----

Para entrenar una red recurrente es necesario que los caracteres se conviertan a una **representación numérica**, en este caso, crearemos dos tablas de consulta, una para mapear los caracteres a enteros y otra para mapear de enteros a caracteres:

In [ ]:
# Convertir de carácter a número entero
char2idx = {u:i for i, u in enumerate(vocab)}

# Mostramos la codificación de los primeros 20 caracteres
print('{')
for char,_ in zip(char2idx, range(20)):
    print(f'  {repr(char):4s}: {char2idx[char]:3d}')
print('  ...\n}')

In [ ]:
# Convertir de entero a String
idx2char = np.array(vocab)
idx2char

In [ ]:
# Convertimos todo el texto a enteros :
text_as_int = np.array([char2idx[c] for c in text])

Veamos un ejemplo de cómo es la representación de los primeros 20 caracteres del texto :

In [ ]:
print(f"Caracteres originales:\n\t{repr(text[10:30])}")
print(f"Codificación:\n\t{text_as_int[10:30]}")

### **2.1.2. Ejemplos y etiquetas**
----

Como dijimos anteriormente, utilizaremos un enfoque *Many-to-Many*, es decir, la entrada y la salida de la red neuronal serán secuencias de tamaño ```seq_length```. Ahora, veamos cómo crear un `Dataset` dentro de _TensorFlow_ con esta estructura :

In [ ]:
# Definimos la longitud de cada secuencia
seq_length = 100
# Definimos el número de ejemplos que verá la red en cada época
examples_per_epoch = len(text) // (seq_length + 1)
# Creamos un dataset con la representación vectorizada
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
# Mostramos un ejemplo con un batch de tamaño 10
for i in char_dataset.take(10):
    print(idx2char[i])

Utilizamos el método `batch` para convertir las secuencias al tamaño que deseamos:

In [ ]:
# Obtenemos las secuencias de tamaño 100 + 1
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
# Mostramos las primeras 5 secuencias
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

Creamos un dataset donde las observaciones son secuencias de tamaño 100 y las etiquetas son la misma secuencia desplazada un carácter, es decir excluye el primer carácter e incluye el carácter siguiente :

In [ ]:
# Creamos una función para separar el último carácter
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# Aplicamos la función a todo el dataset original
dataset = sequences.map(split_input_target)

In [ ]:
dataset

- Veamos un ejemplo :

In [ ]:
for entrada, salida in dataset.take(2):
    print("Entrada:")
    print(repr(''.join(idx2char[entrada.numpy()])))
    print("Salida:")
    print(repr(''.join(idx2char[salida.numpy()])))

Ahora, especificamos el número de _Batches_ que usaremos para entrenar la red y un tamaño de _Buffer_, el cual es necesario para aleatorizar localmente las observaciones (```tf.data``` carga este número de muestras en memoria para poder aleatorizar el entrenamiento).

In [ ]:
batch_size = 64
buffer_size = 10000

# Creamos el dataset con el Batch size y el Buffer
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
dataset

Como podemos ver, cada _batch_ tiene 64 secuencias de tamaño 100 en la entrada y en la salida. La cadena de la salida corresponde un desplazamiento de un carácter hacia adelante de la cadena de entrada.

## **2.2. SimpleRNN**
----

Una capa **SimpleRNN** contiene unidades básicas de neuronas recurrentes, es decir, se trata de **neuronas totalmente conectadas** donde la salida es realimentada a la entrada.

<center><img src='https://drive.google.com/uc?export=view&id=1MX2WYF6JGBZ5OsphTU-C2iAjJK7sQili' alt = "Gráfico ilustrativo del lazo de realimentación sobre una red neuronal " width ="25%"></img></center>

Veamos cómo construir un modelo con SimpleRNN en _TensorFlow_:

### **2.2.1. Definición del modelo**
----

Utilizaremos ```tf.keras.models.Sequential``` para definir el modelo, en este caso utilizaremos tres capas :

* ```tf.keras.layers.Embedding```: Se trata de la entrada del modelo y busca convertir los valores enteros en un *embedding*.
* ```tf.keras.layers.SimpleRNN```: Se trata de la capa con unidades recurrentes.
* ```tf.keras.layers.Dense```: Agregamos una capa totalmente conectada para la predicción del carácter más probable.

La capa `SimpleRNN` se utiliza para modelar secuencias temporales y puede procesar datos secuenciales. Los argumentos más importantes de la capa `SimpleRNN` son:

*    `units`: Este argumento tipo `int` es obligatorio y define el número de unidades recurrentes (neuronas) en la capa SimpleRNN. Aumentar el número de unidades puede aumentar la capacidad del modelo para capturar dependencias temporales más complejas, pero también puede aumentar el tiempo de entrenamiento y el riesgo de sobreajuste.

*    `activation`: Especifica la función de activación que se utilizará en las unidades recurrentes. Por defecto, se usa la función de activación `tanh`.

*    `dropout`, `recurrent_dropout`: Estos argumentos, que varían entre 0 y 1, especifican la tasa de _dropout_ para la entrada y la conexión recurrente, respectivamente.

*    `recurrent_initializer`: Define el método de inicialización de los pesos para la matriz de conexión recurrente en la capa RNN. La matriz de conexión recurrente representa las conexiones entre las unidades recurrentes en pasos de tiempo consecutivos.

*    `return_sequences`: Un valor booleano que indica si la capa debe devolver la secuencia completa de salidas para cada paso de tiempo en lugar de solo la última salida. Por defecto, es `False`. Esto es útil cuando se desea apilar varias capas RNN o cuando se necesita la secuencia completa de salidas para una tarea específica.

*    `return_state`: Un valor booleano que indica si la capa debe devolver el último estado oculto además de la salida. Por defecto, es `False`. Esto puede ser útil si desea utilizar el último estado oculto en otras partes del modelo o si está trabajando con un modelo de secuencia a secuencia.

*    `stateful`: Un valor booleano que indica si la capa debe mantener su estado oculto entre llamadas consecutivas para su uso en secuencias más largas o en lotes con dependencias temporales. Por defecto, es `False`.

Aquí usaremos una capa recurrente con 1024 unidades, y un inicializador `glorot_uniform`. Esta inicialización de pesos tiene como objetivo mantener una varianza adecuada en las salidas y gradientes a lo largo de las capas durante el entrenamiento de una red neuronal, lo que puede ayudar a mejorar la convergencia y la estabilidad del entrenamiento.

In [ ]:
# Dimensión del embedding
embedding_dim = 256
# Número de unidades en la capa recurrente
rnn_units = 1024

model_srnn = tf.keras.Sequential([tf.keras.layers.InputLayer(batch_input_shape=[batch_size, None]),
                                  tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                  tf.keras.layers.SimpleRNN(rnn_units,
                                                            return_sequences=True, # Este argumento hace que el modelo sea Many-to-Many
                                                            stateful=True,
                                                            recurrent_initializer='glorot_uniform'),
                                  tf.keras.layers.Dense(vocab_size)])
model_srnn.summary()

- Vemos el modelo de forma gráfica :

In [ ]:
tf.keras.utils.plot_model(model_srnn)

Es importante resaltar el **tamaño de salida del modelo**, el cual se compone de: ```(batch_size, sequence_length, vocab_size)```, como se puede observar, el modelo no requiere secuencias de un tamaño fijo, por lo cual, puede ser entrenado con una secuencia de un tamaño dado y posteriormente usarse con secuencias de distinta longitud.

- La idea de este modelo es predecir desde el segundo hasta el último carácter de la secuencia de entrada junto con el carácter sucesor. En este caso, nos enfocaremos primordialmente en la distribución de este último.

Veamos las distribuciones predichas por el modelo (pesos aleatorios) :

In [ ]:
# Obtenemos la predicción para el primer Batch
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model_srnn(input_example_batch)
# Seleccionamos la secuencia deseada
print(f"Tamaño de la secuencia de entrada {input_example_batch.shape}")
print(f"Tamaño de la secuencia de salida target {target_example_batch.shape}")
print(f"Tamaño de la secuencia de salida predicha {example_batch_predictions.shape}")


Podemos ver que la predicción para una secuencia de tamaño 100 (codificación en enteros) es una secuencia de tamaño 100 (one-hot).
- Veamos la distribución del carácter sucesor :


In [ ]:
print("Entrada: \n", repr("".join(idx2char[input_example_batch[0]])))

plt.figure(figsize=(20,10))
plt.subplot(211)
plt.bar([repr(i) for i in idx2char], example_batch_predictions[0][-1])
plt.xlim([0, 88])
plt.title("Logits")
plt.subplot(212)
plt.bar([repr(i) for i in idx2char], tf.nn.softmax(example_batch_predictions[0][-1]))
plt.xlim([0, 88])
plt.title("Probits")

### **2.2.2. Entrenamiento**
----

Como nuestro dataset se compone de valores enteros (no estamos usando one-hot encoding para los valores verdaderos, aunque si para las predicciones), utilizaremos la pérdida `tf.keras.losses.sparse_categorical_crossentropy`. Así mismo, como nuestra red no está prediciendo el valor entero directamente (está prediciendo los logits de cada número) especificamos el argumento `from_logits=True`:

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

- Ahora, compilamos el modelo. Usamos `Adam` como optimizador.

In [ ]:
model_srnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                   loss=loss)

- Finalmente, se debe entrenar el modelo.

***Nota: En este caso no entrenaremos el modelo, ya que, el entrenamiento puede llegar a tomar mucho tiempo (para entrenarlo utilice el código comentado). En lugar, cargaremos los pesos de un modelo previamente entrenado.***

In [ ]:
# Utilice el siguiente código si desea entrenar el modelo (puede tomar tiempo)
"""
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath="srnn.h5",
    verbose = 1,
    save_weights_only=True)
model_srnn.fit(dataset.repeat(), epochs=20, callbacks=[checkpoint_callback],
               steps_per_epoch=examples_per_epoch//batch_size, verbose=True)
"""

In [ ]:
# Descargamos los pesos de un modelo preentrenado
!wget -O srnn.h5 https://github.com/mindlab-unal/mlds5-dataset-unit4-RedesNeuronalesRecurrentes/blob/main/srnn.h5?raw=true

In [ ]:
# Cargamos los pesos del modelo
model_srnn.load_weights("srnn.h5")

- Veamos el mismo ejemplo que mostramos al comienzo para mostrar la distribución del carácter sucesor pero con el modelo entrenado :

In [ ]:
# Obtenemos la predicción para el primer Batch
example_batch_predictions = model_srnn(input_example_batch)
# Seleccionamos la secuencia deseada
idx = 6
example_pred_seq = example_batch_predictions[idx]

print("Entrada: \n", repr("".join(idx2char[input_example_batch[idx]])))
print(f"Carácter sucesor más probable: {idx2char[np.argmax(example_pred_seq[-1])]}")
plt.figure(figsize=(20,10))
plt.subplot(211)
plt.bar([repr(i) for i in idx2char], example_pred_seq[-1])
plt.xlim([0, 88])
plt.title("Logits")
plt.subplot(212)
plt.bar([repr(i) for i in idx2char], tf.nn.softmax(example_pred_seq[-1]))
plt.xlim([0, 88])
plt.title("Probits")

## **2.3. Long Short-Term Memory LSTM**
----

Uno de los principales problemas de las redes recurrentes es que **tienden a tener una memoria de corto plazo**, es decir, las redes tenderán a memorizar los últimos ejemplos que vieron mientras olvidan ejemplos vistos anteriormente. Es decir, tienen memoria de corto plazo.

- Una variación de red recurrente son las **long short-term memory (LSTMs)**, se trata de unidades diseñadas para aprender dependencias a largo plazo, al igual que una unidad recurrente también se entrenan en forma de cadena :

<center><img src='https://drive.google.com/uc?export=view&id=1XH-H8bzLnHyD75mke2OWIl32aA23J8r3' alt ="Gráfico ilustrativo de una red neuronal recurrente LSTM" width ="80%"></img></center>

No obstante, cada unidad se compone de **cuatro neuronas** convencionales, cada una aportando un efecto sobre el estado anterior y el estado actual.

- Los círculos rojos con una X que se muestran en la anterior imagen, corresponden a *gate connections*, conexiones que son de tipo multiplicativo. Estas conexiones permiten controlar el **flujo de información** a través del tiempo. Dependiendo del valor que las alimente pueden dejar pasar (un 1) o bloquear (un 0) el paso de información.

<center><img src='https://drive.google.com/uc?export=view&id=1IBIB3n2QBbfCq8hUlCe5dlRxyXBotTgW' alt ="  Gráfico ilustrativo de como se ejecuta la LSTM para generar logits que predicen la probabilidad logarítmica del siguiente carácter" width ="50%"></img></center>

Veamos un ejemplo con una capa LSTM :

### **2.3.1. Definición del modelo**
----
Vamos a definir un modelo equivalente al de la red recurrente simple, sin embargo, ahora agregaremos una **capa LSTM**. Veamos como implementar este modelo :

In [ ]:
# Dimensión del embedding
embedding_dim = 256
# Número de unidades en la capa recurrente
rnn_units = 1024

model_lstm = tf.keras.Sequential([tf.keras.layers.InputLayer(batch_input_shape=[batch_size, None]),
                                  tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                  tf.keras.layers.LSTM(rnn_units,
                                                       return_sequences=True, # Este argumento hace que el modelo sea Many-to-Many
                                                       stateful=True,
                                                       recurrent_initializer='glorot_uniform'),
                                  tf.keras.layers.Dense(vocab_size)])
model_lstm.summary()
tf.keras.utils.plot_model(model_lstm)

Veamos las distribuciones predichas por el modelo (pesos aleatorios) :

In [ ]:
# Obtenemos la predicción para el primer Batch
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model_lstm(input_example_batch)
# Seleccionamos la secuencia deseada
example_pred_seq = example_batch_predictions[0]
print(f"Tamaño de la secuencia de entrada {input_example_batch[0].shape}")
print(f"Tamaño de la secuencia de salida {example_pred_seq.shape}")

- Veamos la distribución del carácter sucesor :

In [ ]:
print("Entrada: \n", repr("".join(idx2char[input_example_batch[0]])))

plt.figure(figsize=(20,10))
plt.subplot(211)
plt.bar([repr(i) for i in idx2char], example_pred_seq[-1])
plt.xlim([0, 88])
plt.title("Logits")
plt.subplot(212)
plt.bar([repr(i) for i in idx2char], tf.nn.softmax(example_pred_seq[-1]))
plt.xlim([0, 88])
plt.title("Probits")

### **2.3.2. Entrenamiento**
----

Seguimos el mismo enfoque que usamos en el modelo anterior, es decir, misma pérdida y mismo optimizador :

In [ ]:
model_lstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss)

- Finalmente, se debe entrenar el modelo.

***Nota: En este caso no entrenaremos el modelo, ya que, el entrenamiento puede llegar a tomar mucho tiempo (para entrenarlo utilice el código comentado). En lugar, cargaremos los pesos de un modelo previamente entrenado.***

In [ ]:
# Utilice el siguiente código si desea entrenar el modelo (puede tomar tiempo)
"""
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath="lstm.h5",
    save_weights_only=True)
model_lstm.fit(dataset.repeat(), epochs=50, callbacks=[checkpoint_callback],
               steps_per_epoch=examples_per_epoch//batch_size)
"""

In [ ]:
# Descargamos los pesos de un modelo preentrenado
!wget -O lstm.h5 https://github.com/mindlab-unal/mlds5-dataset-unit4-RedesNeuronalesRecurrentes/blob/main/lstm.h5?raw=true
model_lstm.load_weights("lstm.h5")

Veamos el mismo ejemplo que mostramos al comienzo para mostrar la distribución del carácter sucesor pero con el modelo entrenado :

In [ ]:
# Obtenemos la predicción para el primer Batch
example_batch_predictions = model_lstm(input_example_batch)
# Seleccionamos la secuencia deseada
example_pred_seq = example_batch_predictions[0]

print("Entrada: \n", repr("".join(idx2char[input_example_batch[0]])))
print(f"Carácter sucesor más probable: {idx2char[np.argmax(example_pred_seq[-1])]}")
plt.figure(figsize=(20,10))
plt.subplot(211)
plt.bar([repr(i) for i in idx2char], example_pred_seq[-1])
plt.xlim([0, 88])
plt.title("Logits")
plt.subplot(212)
plt.bar([repr(i) for i in idx2char], tf.nn.softmax(example_pred_seq[-1]))
plt.xlim([0, 88])
plt.title("Probits")

## **2.4. Gated Recurrent Unit GRU**
----

**Las gated recurrent units (GRUs)** son versiones mejoradas de las redes neuronales recurrentes estándar y son una variante de las LSTM.

- Este tipo de unidades retienen las propiedades de las LSTM siendo más simples y rápidas que las LSTM.

Las GRUs poseen dos compuertas: **_update gate_** y **_reset gate_**, las cuales deciden qué información debe llegar a la salida y si la información se mantiene o se reescribe :

<center><img src='https://drive.google.com/uc?export=view&id=1w6nC0Q5vqR67WQrO8tGQGRBCZsTxlGBm' alt ="Gráfico ilustrativo de la gated recurrent unit" width ="70%"></img></center>


### **2.4.1 Definición del modelo**
----

Definimos un modelo equivalente a los dos modelos anteriores, pero con GRUs:


In [ ]:
# Dimensión del embedding
embedding_dim = 256
# Número de unidades en la capa recurrente
rnn_units = 1024

model_gru = tf.keras.Sequential([tf.keras.layers.InputLayer(batch_input_shape=[batch_size, None]),
                                 tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 tf.keras.layers.GRU(rnn_units,
                                                     return_sequences=True, # Este argumento hace que el modelo sea Many-to-Many
                                                     stateful=True,
                                                     recurrent_initializer='glorot_uniform'),
                                 tf.keras.layers.Dense(vocab_size)])
model_gru.summary()
tf.keras.utils.plot_model(model_gru)

Podemos ver que este modelo tiene un menor número de parámetros en comparación con el modelo de LSTM. Veamos las distribuciones predichas por el modelo (pesos aleatorios) :

In [ ]:
# Obtenemos la predicción para el primer Batch
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model_gru(input_example_batch)
# Seleccionamos la secuencia deseada
example_pred_seq = example_batch_predictions[0]
print(f"Tamaño de la secuencia de entrada {input_example_batch[0].shape}")
print(f"Tamaño de la secuencia de salida {example_pred_seq.shape}")

- Veamos la distribución del carácter sucesor :

In [ ]:
print("Entrada: \n", repr("".join(idx2char[input_example_batch[0]])))

plt.figure(figsize=(20,10))
plt.subplot(211)
plt.bar([repr(i) for i in idx2char], example_pred_seq[-1])
plt.xlim([0, 88])
plt.title("Logits")
plt.subplot(212)
plt.bar([repr(i) for i in idx2char], tf.nn.softmax(example_pred_seq[-1]))
plt.xlim([0, 88])
plt.title("Probits")

### **2.4.2. Entrenamiento**
----

Seguimos el mismo enfoque que usamos con los dos modelos anteriores :

In [ ]:
model_gru.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss)

- Finalmente, se debe entrenar el modelo.

***Nota: En este caso no entrenaremos el modelo, ya que, el entrenamiento puede llegar a tomar mucho tiempo (para entrenarlo utilice el código comentado). En lugar, cargaremos los pesos de un modelo previamente entrenado.***

In [ ]:
# Utilice el siguiente código si desea entrenar el modelo (puede tomar tiempo)
"""
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath="gru.weights.h5",
    save_weights_only=True)
model_gru.fit(dataset.repeat(), epochs=50, callbacks=[checkpoint_callback],
              steps_per_epoch=examples_per_epoch//batch_size)
"""

In [ ]:
# Descargamos los pesos de un modelo preentrenado
!wget -O gru.h5 https://github.com/mindlab-unal/mlds5-dataset-unit4-RedesNeuronalesRecurrentes/blob/main/gru.h5?raw=true

In [ ]:
# Cargamos los pesos del modelo
# Si entrenó el modelo por su cuenta, cambiar a gru.weights.h5
model_gru.load_weights("gru.h5")

Veamos el mismo ejemplo que mostramos al comienzo para mostrar la distribución del carácter sucesor pero con el modelo entrenado :

In [ ]:
# Obtenemos la predicción para el primer Batch
example_batch_predictions = model_gru(input_example_batch)
# Seleccionamos la secuencia deseada
example_pred_seq = example_batch_predictions[0]

print("Entrada: \n", repr("".join(idx2char[input_example_batch[0]])))
print(f"Carácter sucesor más probable: {idx2char[np.argmax(example_pred_seq[-1])]}")
plt.figure(figsize=(20,10))
plt.subplot(211)
plt.bar([repr(i) for i in idx2char], example_pred_seq[-1])
plt.xlim([0, 88])
plt.title("Logits")
plt.subplot(212)
plt.bar([repr(i) for i in idx2char], tf.nn.softmax(example_pred_seq[-1]))
plt.xlim([0, 88])
plt.title("Probits")

## **2.5. Probabilidades de textos**
----
El modelo que fue entrenado permite calcular la probabilidad de un carácter sucesor dada una secuencia de caracteres previos :

- $P(c_i | c_{1},\dots, c_{i-1})$. Podemos utilizar esta probabilidad condicional para calcular la probabilidad conjunta de una secuencia como se muestra a continuación :

$$
P(c_1,\dots,c_n)=P(c_1)\prod_{i=2}^n P(c_i|c_1,\dots,c_{i-1})
$$

Esta función nos permite saber qué tan probable es una secuencia de caracteres según lo que aprendió el modelo. En la práctica, utilizar esta probabilidad conjunta no es muy recomendable (por la inestabilidad numérica que conlleva). Por ello, vamos a utilizar la función de log-verosimilitud del modelo :

$$
L(c_1,\dots,c_n)=\log{P(c_1)}+\sum_{i=2}^N \log{P(c_i|c_1,\dots,c_{i-1})}
$$

- Veamos como definir esta función:

In [ ]:
# Función que calcula la probabilidad de un Texto de acuerdo con la fórmula dada anteriormente:
def log_likelihood(model, text):
    res = []
    for i,char in enumerate(text[:-1]):
        # Creamos un Batch de ceros de secuencias de tamaño 1
        text_int = np.zeros((64,1))
        # Asignamos la codificación del carácter
        text_int[0] = char2idx[char]
        # Obtenemos la distribución del carácter sucesor
        probs = tf.nn.softmax(model.predict(text_int, batch_size=64, verbose=False)[0,-1])
        # Calculamos el log-likelihood del siguiente carácter
        res.append(probs[char2idx[text[i+1]]].numpy())
    return np.log(res).sum()

Veamos un ejemplo de qué oraciones son más probables (mayor log-likelihood) dentro del corpus :

In [ ]:
print(log_likelihood(model_lstm, "la ecuacion del volumen de la esfera"))
print(log_likelihood(model_lstm, "Dios ama a sus hijos los judios "))
print(log_likelihood(model_lstm, "la reina de inglaterra viaja en globo"))

Esto permite detectar cual es la probabilidad de que un texto particular haya sido generado por el mismo proceso que genero el texto de entrenamiento.
- Por ejemplo, el siguiente texto es aún menos probable pues no contiene espacios que separen las palabras:


In [ ]:
print(log_likelihood(model_lstm, "lareinadeinglaterraviajaenglobo"))

También podemos identificar cuándo una palabra o secuencia no tiene una estructura morfológica correcta:

In [ ]:
from itertools import permutations
text = list(u'dios')
# Creamos todas las posibles permutaciones de la palabra dios
perms = [''.join(perm) for perm in permutations(text)]
perms

In [ ]:
text = list(u'dios')
perms = [''.join(perm) for perm in permutations(text)]

# Mostramos las secuencias ordenadas por verosimilitud
for p, t in sorted([(log_likelihood(model_lstm, text), text) for text in perms], reverse=True):
    print(p, t)

## **2.6. Generación automática de texto**

Para la generación automática de texto definimos la función `generate_text`. Esta función realiza los siguientes pasos en cada iteración:

* Comienza codificando un _String_ inicial a enteros.
* Realizar una predicción a partir de la codificación de la entrada.
* Utiliza distribución categórica de la predicción para muestrear el índice (en el vocabulario) del siguiente carácter. Este será la **entrada de la siguiente iteración**.
* Cada que se realiza una predicción, los estados internos (lazos de _Feedback_) se van ajustando hasta que el modelo entiende mejor el contexto y puede realizar mejores predicciones.

<center><img src='https://drive.google.com/uc?export=view&id=1QHKE4ZIP4ELkhjHIHizaOBX61ucfJusR' alt ="Generación de texto con un RNN" width ="80%"></img></center>

Típicamente, en la generación de texto se utiliza un parámetro conocido como _Temperature_. Este es un hiperparámetro que se utiliza para controlar la aleatoriedad y la diversidad en la generación de texto. La temperatura afecta la distribución de probabilidad de las palabras o tokens generados por el modelo durante la decodificación.

Como se acabó de mencionar, la decodificación se realiza generalmente utilizando un enfoque de muestreo o búsqueda de haz (beam search) sobre la distribución de probabilidad de las letras o palabras generadas por el modelo. La temperatura se aplica a esta distribución de probabilidad para ajustar el equilibrio entre la diversidad y la calidad del texto generado.

Cuando la temperatura es igual a 1, se utiliza la distribución original de probabilidad dada por el modelo. A medida que disminuye la temperatura (valores menores a 1), la distribución de probabilidad se vuelve más "puntiaguda", lo que significa que las palabras con mayor probabilidad se vuelven aún más probables, mientras que las palabras con menor probabilidad se vuelven menos probables. Esto puede resultar en una generación de texto más conservadora y centrada en el contexto, pero también puede generar repeticiones y menos diversidad en la salida.

Por otro lado, cuando la temperatura es mayor que 1, la distribución de probabilidad se vuelve más "plana". Esto significa que las palabras menos probables tienen una mayor probabilidad de ser seleccionadas, lo que puede llevar a una generación de texto más creativa y diversa, pero también puede resultar en una salida menos coherente y gramaticalmente incorrecta.


In [ ]:
def generate_text(model, start_string, text_len=1000, temperature=0.5):
    # model: Modelo de tipo Many-to-Many
    # start_string: Secuencia de caracteres inicial
    # text_len: Longitud del texto a generar
    # temperature: Parámetro de control de la distribución categórica
    #               - Valores de temperature bajos (< 1) lleva a valores más determinísticos
    #               - Valores de temperature altos (> 1) aumenta la aleatoriedad.


    # Vectorizamos el String inicial
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Lista para guardar los resultados
    text_generated = []

    # Reiniciamos los estados del modelo
    for layer in model.layers:
        if hasattr(layer, 'reset_states'):
            layer.reset_states()
    # Iteramos para obtener el número de carácteres deseado
    for i in range(text_len):
        # Obtenemos las predicciones
        predictions = model(input_eval)
        # Removemos el eje de los batch
        predictions = tf.squeeze(predictions, 0)
        # Utilizamos la distribución categórica para obtener el siguiente carácter
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        # predicted_id es el carácter predicho (este será la entrada en la siguiente iteración)
        input_eval = tf.expand_dims([predicted_id], 0)
        # Agregamos el String correspondiente al id predicho
        text_generated.append(idx2char[predicted_id])
    return (start_string + ''.join(text_generated))

Ahora, veamos el texto generado con cada uno de los modelos entrenados:

* **SimpleRNN**:

In [ ]:
# Creamos un modelo con un tamaño de batche 1 con los pesos del modelo entrenado
model_srnn = tf.keras.Sequential([tf.keras.layers.InputLayer(batch_input_shape=[1, None]),
                                  tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                  tf.keras.layers.SimpleRNN(rnn_units,
                                                            return_sequences=True, # Este argumento hace que el modelo sea Many-to-Many
                                                            stateful=True,
                                                            recurrent_initializer='glorot_uniform'),
                                  tf.keras.layers.Dense(vocab_size)])

model_srnn.load_weights("srnn.h5")

In [ ]:
model_srnn.summary()

- Comenzamos generando texto con un valor de temperatura bajo :

In [ ]:
print(generate_text(model_srnn, start_string=u"Dios dijo: ", temperature=0.1))

- Ahora, veamos el resultado con un valor de temperatura alto :

In [ ]:
print(generate_text(model_srnn, start_string=u"Dios dijo: ", temperature=2.0))

Lo ideal es encontrar un valor de temperatura apropiado, es decir, donde el texto generado no sea tan repetitivo o determinístico ni tan aleatorio e incoherente, veamos un ejemplo con un valor apropiado :

In [ ]:
print(generate_text(model_srnn, start_string=u"Dios dijo: ", temperature=0.5))

* **LSTM**:

In [ ]:
# Creamos un modelo con un tamaño de batche 1 con los pesos del modelo entrenado
model_lstm = tf.keras.Sequential([tf.keras.layers.InputLayer(batch_input_shape=[1, None]),
                                  tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                  tf.keras.layers.LSTM(rnn_units,
                                                       return_sequences=True, # Este argumento hace que el modelo sea Many-to-Many
                                                       stateful=True,
                                                       recurrent_initializer='glorot_uniform'),
                                  tf.keras.layers.Dense(vocab_size)])

model_lstm.load_weights("lstm.h5")

In [ ]:
model_lstm.summary()

In [ ]:
print(generate_text(model_lstm, start_string=u"Dios dijo: "))

* **GRU**:

In [ ]:
# Creamos un modelo con un tamaño de batche 1 con los pesos del modelo entrenado
model_gru = tf.keras.Sequential([tf.keras.layers.InputLayer(batch_input_shape=[1, None]),
                                 tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 tf.keras.layers.GRU(rnn_units,
                                                     return_sequences=True, # Este argumento hace que el modelo sea Many-to-Many
                                                     stateful=True,
                                                     recurrent_initializer='glorot_uniform'),
                                 tf.keras.layers.Dense(vocab_size)])

model_gru.load_weights("gru.h5")

In [ ]:
model_gru.summary()

In [ ]:
print(generate_text(
    model_gru, start_string=u"El sentido de la vida es ", temperature=0.3,
    text_len=10000
    ).strip())

# **Conclusión**

A lo largo de este notebook, hemos explorado y analizado el funcionamiento, las aplicaciones y las técnicas relacionadas con las redes neuronales recurrentes (RNN). Las RNN son una clase de modelos de aprendizaje profundo diseñados para procesar secuencias temporales o datos secuenciales, como series de tiempo, texto y señales de audio. Las RNN tienen la capacidad de mantener información de estado a lo largo del tiempo, lo que les permite capturar dependencias temporales y contextuales en los datos.

# **Recursos adicionales**
----
* [*Fundamentals of Deep Learning – Introduction to Recurrent Neural Networks*](https://www.analyticsvidhya.com/blog/2017/12/introduction-to-recurrent-neural-networks/?utm_source=blog&utm_medium=best-papers-iclr-2019)
* [*Understanding LSTM Networks*](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [*Generación de texto con un RNN - Construir el modelo*](https://www.tensorflow.org/tutorials/text/text_generation#build_the_model)

* _Origen de los íconos_

    - Generación de texto con un RNN. (n.d.). TensorFlow [Imagen] https://www.tensorflow.org/text/tutorials/images/text_generation_sampling.png?hl=es-419

# **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes :**
  * [Santiago Toledo Cortés](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
  * [Juan Sebastián Lara](https://http://juselara.com/)
* **Diseño de imágenes:**
    - [Mario Andres Rodriguez Triana](https://www.linkedin.com/in/mario-andres-rodriguez-triana-394806145/).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*